In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# define the path to the dataset directory
DATASET_PATH = 'CS552J_DMDL_Assessment_1_Dataset'

# define the size of the images
IMG_SIZE = (224, 224)

# define the number of channels
CHANNELS = 3

# define the label categories
CATEGORIES = ['Covid-19', 'Normal']

# initialize the data and labels arrays
data = []
labels = []

# loop over the image paths
for category in CATEGORIES:
    path = os.path.join(DATASET_PATH, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        image = cv2.resize(image, IMG_SIZE)
        image = np.array(image, dtype=np.float32)
        image /= 255.0
        data.append(image)
        labels.append(category)

# convert the data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# split the dataset into training, validation, and testing sets
(train_data, test_data, train_labels, test_labels) = train_test_split(data, labels, test_size=0.2, random_state=42)
(train_data, val_data, train_labels, val_labels) = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

In [2]:
print(train_labels)

['Covid-19' 'Covid-19' 'Covid-19' 'Normal' 'Covid-19' 'Covid-19' 'Normal'
 'Normal' 'Covid-19' 'Normal' 'Normal' 'Covid-19' 'Covid-19' 'Covid-19'
 'Normal' 'Covid-19' 'Normal' 'Covid-19' 'Covid-19' 'Normal' 'Normal'
 'Covid-19' 'Covid-19' 'Normal' 'Covid-19' 'Normal' 'Normal' 'Covid-19'
 'Normal' 'Normal' 'Normal' 'Covid-19' 'Covid-19' 'Normal' 'Normal'
 'Normal' 'Covid-19' 'Covid-19' 'Covid-19' 'Normal' 'Normal' 'Normal'
 'Normal' 'Covid-19' 'Covid-19' 'Covid-19' 'Normal' 'Covid-19' 'Normal'
 'Covid-19' 'Normal' 'Normal' 'Covid-19' 'Normal' 'Normal' 'Covid-19'
 'Covid-19' 'Covid-19' 'Normal' 'Covid-19' 'Normal' 'Covid-19' 'Covid-19'
 'Normal' 'Normal' 'Covid-19' 'Normal' 'Covid-19' 'Normal' 'Covid-19'
 'Covid-19' 'Covid-19' 'Normal' 'Covid-19' 'Covid-19' 'Normal' 'Normal'
 'Covid-19' 'Normal' 'Normal' 'Normal' 'Normal' 'Covid-19' 'Normal'
 'Normal' 'Normal' 'Covid-19' 'Normal' 'Covid-19' 'Normal' 'Normal'
 'Covid-19' 'Covid-19' 'Covid-19' 'Normal' 'Normal' 'Covid-19' 'Covid-19'
 'Norm

In [3]:
# define label encoder
label_encoder = {'Covid-19': 1, 'Normal': 0}

# convert labels to numerical values
train_labels = np.array([label_encoder[label] for label in train_labels])
val_labels = np.array([label_encoder[label] for label in val_labels])
test_labels = np.array([label_encoder[label] for label in test_labels])

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

from tensorflow.keras.optimizers import Adam

In [7]:
# GPU check
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.gpu_device_name())

Num GPUs Available:  1
/device:GPU:0


In [9]:
# Set memory growth to true
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
import tensorflow as tf

# check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Limit GPU memory usage to avoid OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            print(gpu)
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [10]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Function to create model
def create_model(lr=0.001, dropout_rate=0.0, l2=0.0):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3), kernel_regularizer=regularizers.l2(l2)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(l2)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(lr=lr)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create model
model = KerasClassifier(build_fn=create_model)

# Define hyperparameters and their values
params = {'lr': [0.1, 0.01, 0.001], 'dropout_rate': [0.0, 0.25, 0.5], 'l2': [0.0, 0.001, 0.01]}

# Create grid search object
grid = GridSearchCV(estimator=model, param_grid=params, cv=3)

# Fit grid search object to the data
grid_result = grid.fit(train_data, train_labels)

# Print results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


c:\Users\abhij\.conda\envs\image_ML\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
81 fits failed out of a total of 81.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\abhij\.conda\envs\image_ML\lib\site-packages\tensorflow\python\client\session.py", line 1356, in _do_call
    return fn(*args)
  File "c:\Users\abhij\.conda\envs\image_ML\lib\site-packages\tensorflow\python\client\session.py", line 1341, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "c:\Users\abhij\.conda\envs\image_ML\lib\site-packages\tensorflow\python\client\session.py", line 1429, in _call_tf_sessionrun
    run

ResourceExhaustedError: OOM when allocating tensor of shape [512] and type float
	 [[node dense_490/bias/Initializer/zeros (defined at C:\Users\abhij\AppData\Local\Temp\ipykernel_29236\3374643717.py:16) ]]

Original stack trace for 'dense_490/bias/Initializer/zeros':
  File "c:\Users\abhij\.conda\envs\image_ML\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\Users\abhij\.conda\envs\image_ML\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\abhij\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\abhij\AppData\Roaming\Python\Python37\site-packages\traitlets\config\application.py", line 982, in launch_instance
    app.start()
  File "C:\Users\abhij\AppData\Roaming\Python\Python37\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "C:\Users\abhij\AppData\Roaming\Python\Python37\site-packages\tornad

In [ ]:
# evaluate CNN model2
test_loss_cnn, test_acc_cnn = model_cnn2.evaluate(test_data, test_labels)

print('Test accuracy:', test_acc_cnn)
print('Test loss:', test_loss_cnn)

40/40 [==============================] - 0s 7ms/sample - loss: 0.3690 - acc: 1.0000
Test accuracy: 1.0
Test loss: 0.3690335929393768


In [16]:
# CNN model0

model_cnn = Sequential()
model_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model_cnn.add(MaxPooling2D((2, 2)))
model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D((2, 2)))
model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D((2, 2)))
model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D((2, 2)))
model_cnn.add(Flatten())
model_cnn.add(Dense(512, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(1, activation='sigmoid'))

model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_cnn = model_cnn.fit(train_data, train_labels, batch_size=32, epochs=50, validation_data=(val_data, val_labels))

Train on 128 samples, validate on 32 samples
Epoch 1/50
128/128 [==============================] - 3s 22ms/sample - loss: 1.1122 - acc: 0.5156 - val_loss: 0.6726 - val_acc: 0.5312
Epoch 2/50
128/128 [==============================] - 0s 3ms/sample - loss: 0.6576 - acc: 0.5234 - val_loss: 0.6252 - val_acc: 0.5625
Epoch 3/50
128/128 [==============================] - 0s 3ms/sample - loss: 0.4872 - acc: 0.8281 - val_loss: 0.3421 - val_acc: 0.9062
Epoch 4/50
128/128 [==============================] - 0s 3ms/sample - loss: 0.2228 - acc: 0.9297 - val_loss: 0.3272 - val_acc: 0.9375
Epoch 5/50
128/128 [==============================] - 0s 2ms/sample - loss: 0.1329 - acc: 0.9688 - val_loss: 0.4964 - val_acc: 0.8750
Epoch 6/50
128/128 [==============================] - 0s 2ms/sample - loss: 0.1296 - acc: 0.9688 - val_loss: 0.5394 - val_acc: 0.8438
Epoch 7/50
128/128 [==============================] - 0s 3ms/sample - loss: 0.1085 - acc: 0.9766 - val_loss: 0.3982 - val_acc: 0.8438
Epoch 8/50
128/1

In [15]:
# evaluate CNN model2
test_loss_cnn, test_acc_cnn = model_cnn.evaluate(test_data, test_labels)

print('Test accuracy:', test_acc_cnn)
print('Test loss:', test_loss_cnn)

40/40 [==============================] - 0s 3ms/sample - loss: nan - acc: 0.5250
Test accuracy: 0.525
Test loss: nan


In [28]:
model = Model(inputs=resnet.input, outputs=output)
model.summary()

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_15[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [12]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model

model = Sequential()

model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='softmax'))

model.layers[0].trainable = False

#I can check the summary of the final Model which we created by using the line of code below.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
flatten_3 (Flatten)          (None, 2048)              0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_7 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
batch_normalization_4 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_5 (Batch (None, 1024)             

In [13]:
lr = 1e-3
epochs = 50
bs = 8
optimizer = Adam(lr = lr, decay= lr/epochs)
model.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
import time
from tensorflow.keras.callbacks import ModelCheckpoint

# Time to train our model !
epochs = 100

#initialize the training data augmentation object
train_datagen = ImageDataGenerator(
        rotation_range=15,
        fill_mode ="nearest")
 
checkpointer = ModelCheckpoint(filepath = "CDX_Best_RestNet50.h5", save_best_only = True, verbose=1)
start = time.time()
 
# let's get started !
 
history=model.fit_generator(train_datagen.flow(train_data, train_labels, batch_size = bs),
                            steps_per_epoch = len(train_data)//bs,
                            validation_data = (val_data, val_labels),
                            validation_steps = len(val_data)//bs,
                            epochs =epochs,
                            callbacks= [checkpointer])
 
end = time.time()
duration = end - start
print ('\n This Model took %0.2f seconds (%0.1f minutes) to train for %d epochs'%(duration, duration/60, epochs) )

Epoch 1/100
14/16 [=========================>....] - ETA: 0s - loss: 7.2559 - acc: 0.5268
Epoch 00001: val_loss improved from inf to 8.14578, saving model to CDX_Best_RestNet50.h5
16/16 [==============================] - 9s 589ms/step - loss: 7.6666 - acc: 0.5000 - val_loss: 8.1458 - val_acc: 0.4688
Epoch 2/100
15/16 [===========================>..] - ETA: 0s - loss: 7.7944 - acc: 0.4917
Epoch 00002: val_loss did not improve from 8.14578
16/16 [==============================] - 1s 70ms/step - loss: 7.6666 - acc: 0.5000 - val_loss: 8.1458 - val_acc: 0.4688
Epoch 3/100
15/16 [===========================>..] - ETA: 0s - loss: 7.5388 - acc: 0.5083
Epoch 00003: val_loss did not improve from 8.14578
16/16 [==============================] - 1s 84ms/step - loss: 7.6666 - acc: 0.5000 - val_loss: 8.1458 - val_acc: 0.4688
Epoch 4/100
15/16 [===========================>..] - ETA: 0s - loss: 7.9222 - acc: 0.4833
Epoch 00004: val_loss did not improve from 8.14578
16/16 [=============================

In [16]:
(eval_loss, eval_accuracy) = model.evaluate(  
     test_data, test_labels, batch_size=bs, verbose=2)
 
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

40/40 - 1s - loss: 7.2833 - acc: 0.5250
Accuracy: 52.50%
Loss: 7.283288669586182


In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# load the pre-trained ResNet-50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# freeze the weights of the pre-trained model
for layer in resnet.layers:
    layer.trainable = False

# add new trainable layers
x = resnet.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# create the new model
model = Model(inputs=resnet.input, outputs=output)a

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
history = model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))

Train on 128 samples, validate on 32 samples
Epoch 1/10
128/128 [==============================] - 2s 19ms/sample - loss: 8.6487 - acc: 0.7266 - val_loss: 31.6129 - val_acc: 0.4688
Epoch 2/10
128/128 [==============================] - 1s 5ms/sample - loss: 1.7779 - acc: 0.9844 - val_loss: 45.2528 - val_acc: 0.4688
Epoch 3/10
128/128 [==============================] - 1s 5ms/sample - loss: 0.0677 - acc: 0.9922 - val_loss: 50.7756 - val_acc: 0.4688
Epoch 4/10
128/128 [==============================] - 1s 5ms/sample - loss: 1.2130e-20 - acc: 1.0000 - val_loss: 52.4058 - val_acc: 0.4688
Epoch 5/10
128/128 [==============================] - 1s 5ms/sample - loss: 6.4832e-20 - acc: 1.0000 - val_loss: 53.3908 - val_acc: 0.4688
Epoch 6/10
128/128 [==============================] - 1s 5ms/sample - loss: 1.6829e-04 - acc: 1.0000 - val_loss: 54.3610 - val_acc: 0.4688
Epoch 7/10
128/128 [==============================] - 1s 5ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 55.0092 - val_acc: 

In [6]:
# evaluate the model
_, train_acc = model.evaluate(train_data, train_labels, verbose=0)
_, val_acc = model.evaluate(val_data, val_labels, verbose=0)
_, test_acc = model.evaluate(test_data, test_labels, verbose=0)
print('Training accuracy: %.3f, Validation accuracy: %.3f, Test accuracy: %.3f' % (train_acc, val_acc, test_acc))

Training accuracy: 0.500, Validation accuracy: 0.469, Test accuracy: 0.525


In [30]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# load the pre-trained ResNet-50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# freeze the weights of the pre-trained model
for layer in resnet.layers:
    layer.trainable = False

# add new trainable layers
x = resnet.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# create the new model
model = Model(inputs=resnet.input, outputs=output)

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
history = model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))

Train on 128 samples, validate on 32 samples


ResourceExhaustedError: OOM when allocating tensor with shape[100352,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dense_28/kernel/Initializer/random_uniform/RandomUniform (defined at C:\Users\abhij\AppData\Local\Temp\ipykernel_9572\2826195802.py:15) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'dense_28/kernel/Initializer/random_uniform/RandomUniform':
  File "c:\Users\abhij\.conda\envs\image_ML\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\Users\abhij\.conda\envs\image_ML\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\abhij\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\abhij\AppData\Roaming\Python\Python37\site-packages\traitlets\co